In [1]:
import obspy
from obspy import read
import numpy as np
import os

In [2]:
#Make folder for AI ready data outside of tracked directory
filepath = os.path.abspath(os.getcwd()).split('/')[0:-2]
filepath = '/'.join(filepath)
print(filepath)

#actually creating directory
data_dir = filepath+'/data'
clean_data_dir = data_dir+'/clean'
aiready_data_dir = data_dir+'/aiready'

os.mkdir(aiready_data_dir)

/Users/atsparks/Desktop/Github


In [3]:
#read in cleaned mseed files, save as daily npy files and one giant 100 day file
hundred_data = np.array([])

for i in range(100):
    i += 1

    #read mseed
    st = read(clean_data_dir+'/'+str(i)+'_cleaned.mseed')

    #save data to day long npys
    data = st[0].data[0:-1] #removing extra point from decimating
    np.save(aiready_data_dir+'/'+str(i)+'_ready.npy', data)

    #append to overall array
    hundred_data = np.append(hundred_data, data)

#save hundred day data npy
np.save(aiready_data_dir+'/100_days.npy', hundred_data)